In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from edgedroid.models.timings import *
import edgedroid.data as default_data

exec_time_data, neuroticism_data, imp_data, dur_data = default_data.load_default_exec_time_data()

data = preprocess_data(exec_time_data, neuroticism_data, imp_data, dur_data)
data

In [2]:
import itertools
from collections import deque
import pandas as pd
from typing import Tuple, Deque
from tqdm.notebook import tqdm

result_rows = deque()

for run_id in tqdm(data["run_id"].unique(), desc="Run"):
    test_data = data[data["run_id"] == run_id].copy()

    def get_test_data(*args, **kwargs) \
            -> Tuple[pd.DataFrame, pd.arrays.IntervalArray, pd.arrays.IntervalArray, pd.arrays.IntervalArray]:
        etime_data = exec_time_data[exec_time_data["run_id"] != run_id].copy()
        return etime_data, neuroticism_data, imp_data, dur_data

    class EmpiricalTestModel(EmpiricalETM):
        get_data = get_test_data

    class TheoreticalTestModel(FittedETM):
        get_data = get_test_data


    neuroticism = test_data["neuroticism_raw"].unique()[0]

    models: Deque[Tuple[str, ExecutionTimeModel, int, int]] = deque()

    for win_size, ttf_bins in itertools.product(range(1, 16), range(1, 10)):
        emp_model = EmpiricalTestModel(neuroticism=neuroticism,
                                       ttf_levels=ttf_bins,
                                       cleanup=CleanupMode.TRUNCATE,
                                       window=win_size)
        theo_model = TheoreticalTestModel(neuroticism=neuroticism,
                                          ttf_levels=ttf_bins,
                                          cleanup=CleanupMode.TRUNCATE,
                                          window=win_size)

        models.extend((
            ("empirical", emp_model, win_size, ttf_bins),
            ("theoretical", theo_model, win_size, ttf_bins)
        ))

    for _ in range(30):
        for i, (prev_ttf, exec_time) in enumerate(test_data[["ttf", "next_exec_time"]].itertuples(index=False)):
            for model_name, model, win_size, ttf_bins in models:
                if i == 0:
                    # reset the model between runs
                    model.reset()
                else:
                    # first row has no previous ttf
                    model.advance(prev_ttf)

                prediction = model.get_execution_time()
                result_rows.append({
                    "real": exec_time,
                    "prediction": prediction,
                    "error": prediction - exec_time,
                    "model": model_name,
                    "window": win_size,
                    "ttf_bins": ttf_bins
                })

results = pd.DataFrame(result_rows)
results

In [3]:
results["sqr_error"] = np.square(results["error"])
results

In [4]:
# %matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

fg = sns.catplot(results, kind="point", x="window", y="sqr_error", hue="ttf_bins", row="model", height=5, aspect=2)
fg.set(ylim=(0, None))
fg.set_ylabels("MSE")
for ax in fg.axes.flat:
    ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
plt.show()